In [ ]:
import robotic as ry
import numpy as np
from visual import getObject, lookAtObj, scanObject, piece_pci
from arena import *
import open3d as o3d
import random

INITIAL_OBJ_POS = [-.5, 0, .69]
ON_REAL = False
ARENA_POS = np.array([-.54, -.17, .651])

GRIPPER_FINGER_SEPARATION = .075
GRIPPER_FINGER_WIDTH = .025

def validSideDistance(p1, p2, n):
    dist = np.linalg.norm(p2-p1)
    normal_dist = abs(np.dot(p1 - p2, n) / np.linalg.norm(n))
    side_dist = np.sqrt(dist*dist-normal_dist*normal_dist)
    return side_dist <= GRIPPER_FINGER_WIDTH*.5

def validNormalDistance(p1, p2, n, dist_thresh=GRIPPER_FINGER_SEPARATION):
    return (abs(np.dot(p1 - p2, n) / np.linalg.norm(n)) <= dist_thresh
            and abs(np.dot(p1 - p2, n) / np.linalg.norm(n)) >= GRIPPER_FINGER_WIDTH)

def validNormalRelativeDir(p1, p2, n):
    p2pvec = p2-p1
    dot_product = np.dot(n, p2pvec)
    # Normal vector has magnitude one so we ignore it
    magnitude_vector = np.linalg.norm(p2pvec)
    cosine_angle = dot_product / magnitude_vector
    return cosine_angle <= 0

def validNormalDir(n1, n2):
    upvec = np.array([0., 0., 1.])
    dot_product1 = np.dot(n1, upvec)
    dot_product2 = np.dot(n2, upvec)
    return dot_product1 >= 0 and dot_product2 >= 0

def findPossibleGrasps(point_cloud, verbose=0):

    # Prepare point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    pcd = pcd.voxel_down_sample(voxel_size=0.005)
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    
    # Find antipodal pairs
    antipodal_pairs = []
    for i in range(len(pcd.points)):
        normal_i = np.asarray(pcd.normals[i])
        for j in range(i + 1, len(pcd.points)):
            normal_j = np.asarray(pcd.normals[j])
            dot_product = np.dot(normal_i, normal_j)
            if (dot_product < -0.7 and
                validSideDistance(pcd.points[i], pcd.points[j], pcd.normals[i]) and
                validNormalDistance(pcd.points[i], pcd.points[j], pcd.normals[i]) and
                validNormalRelativeDir(pcd.points[i], pcd.points[j], pcd.normals[i])
                and validNormalDir(pcd.points[j], pcd.normals[i])
                ):
                    antipodal_pairs.append((i, j))

    if verbose:
        print("Antipodal pairs found:", antipodal_pairs)

        if verbose > 1:
            # Create a line set to visualize antipodal pairs
            line_set = o3d.geometry.LineSet()

            # Populate line_set with antipodal pairs
            lines = []
            for pair in antipodal_pairs:
                lines.append([pair[0], pair[1]])
            lines = np.array(lines).astype(np.int32)
            line_set.points = o3d.utility.Vector3dVector(np.asarray(pcd.points))
            line_set.lines = o3d.utility.Vector2iVector(lines)

            # Set color of the lines to red
            line_colors = np.array([[1, 0, 0] for _ in range(len(lines))])
            line_set.colors = o3d.utility.Vector3dVector(line_colors)

            # Visualize the point cloud and antipodal pairs
            o3d.visualization.draw_geometries([pcd, line_set], point_show_normal=True)

    return antipodal_pairs

In [ ]:
#-- load parameters, typically automatically from 'rai.cfg'
ry.params_print()
verbose = 0

#-- define a configuration
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))

obj = C.addFrame('obj')
obj.setShape(ry.ST.ssBox, size=[.05, .09, .04, .005])
obj.setPosition(INITIAL_OBJ_POS)
obj.setColor([1, .5, .5])

C.addFrame('predicted_obj') \
    .setShape(ry.ST.marker, size=[.1]) \
    .setPosition(INITIAL_OBJ_POS) \
    .setColor([1, 0, 0])

for i in range(32):
    C.addFrame(f'view_point_{i}') \
        .setPosition([0, 0, 0]) \
        .setShape(ry.ST.sphere, size=[.02]) \
        .setColor([1, 1, 0])

bot = ry.BotOp(C, ON_REAL)

obj_pos = INITIAL_OBJ_POS

# Generate Arena
arena = CircularArena(middleP=ARENA_POS, innerR=None, outerR=.5)
arena.plotArena(C)

In [ ]:
# Point towards set initial object position
lookAtObj(bot, C, np.array(INITIAL_OBJ_POS))

# Capture midpoint from point cloud
obj_pos  = getObject(bot, C, arena) 

scanned_views_ = scanObject(bot, C, np.array(obj_pos), arena, view_count=8)
scanned_views = []
for sv in scanned_views_:
    point_cloud_obj = o3d.geometry.PointCloud()
    point_cloud_obj.points = o3d.utility.Vector3dVector(sv)
    scanned_views.append(point_cloud_obj)

In [ ]:
final_points = piece_pci(scanned_views)

In [ ]:
pclFrame = C.getFrame("pcl")
pclFrame.setPointCloud(np.asarray(final_points.points))
C.view_recopyMeshes()

In [ ]:
point_indices = findPossibleGrasps(final_points, verbose=2)

In [ ]:
random_grasp_indices = random.choice(point_indices)
gripping_pos = (final_points.points[random_grasp_indices[0]] + final_points.points[random_grasp_indices[1]]) * .5
gripping_angle = final_points.points[random_grasp_indices[0]] + final_points.points[random_grasp_indices[1]]
gripping_angle /= np.linalg.norm(gripping_angle)

In [ ]:
bot.gripperOpen(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

bot.sync(C)

komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(1., 1, 1., 0)

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)

komo.addObjective([1.], ry.FS.positionDiff, ['l_gripper'], ry.OT.eq, [1e1], gripping_pos)
komo.addObjective([1.], ry.FS.vectorX, ["l_gripper"], ry.OT.eq, [1e1], gripping_angle)
komo.addObjective([1.], ry.FS.vectorZ, ["l_gripper"], ry.OT.eq, [1e1], [0., 0., -1.])

ret = ry.NLP_Solver().setProblem(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=0).solve()
print(ret)
bot.move(komo.getPath(), [3.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

bot.gripperClose(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

bot.home(C)
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

bot.gripperOpen(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

In [ ]:
bot.home(C)
C.view(True)